# Add injury to insult
**A case study of murdering someone with the right choice of words**

### Attacker
Nyah, level 5 witch (The Resentment)
**Skills** Diplomacy +14 (Bon Mot)

**Occult Spells** DC 21; **3rd** Blindness or Paralyze, Biting Words; **2nd** Blistering Invective; **1st** Sure Strike x3 ; **Cantrips (3rd)** Evil Eye

### Attack routine
1. Bon Mot, Blistering Invective
2. Blindness or Paralyze, Evil Eye
3. Evil Eye or Sure Strike, Biting Words
4. Evil Eye, Sure Strike, Biting Words attack
5. Evil Eye, Sure Strike, Biting Words attack

### Assumptions
- The target attempts to clear neither Bon Mot nor Sickened
- No movement is needed; the target remains within 30ft at all time
- Spellcasting is not disrupted or obstructed in any way
- Ignoring damage dealt to other creatures by casting heightened blistering invective

In [ ]:
import numpy as np
import xarray
from pathfinder2e_stats import *

np.random.seed(0)

diplomacy = 14
spell_DC = 21

# You can change any of these to upcast or downcast them;
# damage and incapacitation trait are adjusted automatically
blistering_invective_rank = 2
blindness_paralyze_rank = 3
biting_words_rank = 3

### Targets

In [ ]:
targets = xarray.Dataset(
    {
        "target": [
            "The Stag Lord",
            "Ettin",
            "Vampire Count",
            "Hill Giant",
            "Dweomercat",
            "Sphinx",
        ],
        "level": ("target", [6, 6, 6, 7, 7, 8]),
        "HP": ("target", [110, 110, 65, 140, 100, 135]),
        "AC": ("target", [23, 21, 24, 24, 25, 27]),
        "Will": ("target", [9, 12, 17, 13, 17, 19]),
        "bonus_save_vs_magic": ("target", [0, 0, 0, 0, 1, 0]),
        "sickened": ("target", [1, 0, 0, 0, 0, 0]),
    }
)
targets["rank"] = level2rank(targets.level)
targets.to_pandas()

### Round 1: Bon Mot -> Blistering Invective

In [ ]:
bon_mot = check(diplomacy, DC=targets.Will + 10 - targets.sickened)
bon_mot["Will_penalty"] = map_outcome(
    bon_mot.outcome,
    {DoS.success: 2, DoS.critical_success: 3},
)
_ = bon_mot.Will_penalty.to_pandas().hist(figsize=(10, 8))

In [ ]:
sickened = [targets.sickened]
will = [
    sum_bonuses(
        ("untyped", targets.Will),
        ("status", targets.bonus_save_vs_magic),
        ("status", -targets.sickened),
        ("status", -bon_mot.Will_penalty),
    )
]

blistering_invective = damage(
    check(will[0], DC=spell_DC),
    Damage(
        "fire", blistering_invective_rank // 2 * 2, 6, persistent=True, basic_save=True
    ),
    persistent_damage_rounds=5,
).rename({"persistent_round": "round"})

blistering_invective_damage = (
    blistering_invective["persistent_damage"]
    .where(blistering_invective["apply_persistent_damage"], 0)
    .sum("damage_type")
)
blistering_invective_damage.mean("roll").to_pandas().T

In [ ]:
frightened = map_outcome(
    blistering_invective["outcome"],
    {DoS.failure: 1, DoS.critical_failure: 2},
)
frightened = np.maximum(0, frightened - blistering_invective["round"])
frightened.isel(roll=3).to_pandas()

In [ ]:
_ = frightened.isel(round=0).to_pandas().hist(figsize=(10, 8))

### Round 2: Blindness or Paralyze -> Evil Eye
Blindness and Paralyze are rank 3+ spells with the incapacitation trait. level 7+ targets get the success of their saves one step better. Run the simulation from now on with either of the options.
Evil Eye is used to extend the duration in both cases.

In [ ]:
will.append(
    sum_bonuses(
        ("untyped", targets.Will),
        ("status", targets.bonus_save_vs_magic),
        ("status", -sickened[-1]),
        ("status", -bon_mot.Will_penalty),
        ("status", -frightened.isel(round=1, drop=True)),
    )
)
blindness_paralyze = check(
    bonus=will[-1],
    DC=spell_DC,
    incapacitation=targets["rank"] > blindness_paralyze_rank,
)

# In case of simple success, we use Evil Eye to extend the blindness for the whole combat
blindness_paralyze["need_evil_eye"] = xarray.concat(
    [
        blindness_paralyze.outcome == DoS.success,
        blindness_paralyze.outcome == DoS.failure,
    ],
    dim="incapacitation_spell",
)
blindness_paralyze["off_guard"] = xarray.concat(
    [
        blindness_paralyze.outcome < DoS.critical_success,
        blindness_paralyze.outcome < DoS.success,
    ],
    dim="incapacitation_spell",
)
blindness_paralyze["incapacitation_spell"] = ["blindness", "paralyze"]

In [ ]:
(
    blindness_paralyze[["off_guard", "need_evil_eye"]]
    .mean("roll")
    .to_array("condition")
    .stack(col=["condition", "incapacitation_spell"])
    .to_pandas()
)

In [ ]:
def evil_eye(will_bonus, spell_DC, do_cast=True):
    c = check(will_bonus, DC=spell_DC).outcome
    c = c.where(do_cast, DoS.no_roll)
    return map_outcome(c, {DoS.critical_failure: 2, DoS.failure: 1})


sickened.append(np.maximum(sickened[-1], evil_eye(will[-1], spell_DC)))

In [ ]:
_ = sickened[-1].to_pandas().T.hist(figsize=(10, 8))

### Round 3: Evil Eye or Sure Strike -> Biting Words
If the target scored a simple success vs. Blindness in round 2, extend its duration with Evil Eye.
Otherwise, cast Sure Strike. Then, cast Biting Words.
### Round 4 and 5: Evil Eye -> Sure Strike -> Biting Words attack

In [ ]:
for rnd in range(2, 5):
    will.append(
        sum_bonuses(
            ("untyped", targets.Will),
            ("status", targets.bonus_save_vs_magic),
            ("status", -sickened[-1]),
            ("status", -bon_mot.Will_penalty),
        )
    )
    cast_evil_eye = blindness_paralyze.need_evil_eye if rnd == 2 else True
    sickened.append(
        np.maximum(sickened[-1], evil_eye(will[-1], spell_DC, do_cast=cast_evil_eye))
    )

assert len(will) == 5
assert len(sickened) == 5

will = xarray.concat(will, dim="round")
sickened = xarray.concat(sickened, dim="round")

In [ ]:
off_guard = xarray.concat(
    [
        xarray.DataArray(False),
        blindness_paralyze.off_guard.expand_dims(round=4),
    ],
    dim="round",
)
AC = sum_bonuses(
    ("untyped", targets.AC),
    ("status", -frightened),
    ("status", -sickened),
    ("circumstance", off_guard.astype(int) * -2),
)
AC.mean("roll").stack(col=["round", "incapacitation_spell"]).to_pandas()

In [ ]:
# TODO allow check(... fortune=...) with DataArray parameter
biting_words_check = check(spell_DC - 10, DC=AC, dims={"fortune": 2})
biting_words_check["outcome"] = xarray.concat(
    [
        xarray.DataArray([DoS.no_roll, DoS.no_roll], dims=["round"]),
        xarray.where(
            blindness_paralyze.need_evil_eye,
            biting_words_check["outcome"].isel(round=2, fortune=0),
            biting_words_check["outcome"].isel(round=2).max("fortune"),
        ),
        biting_words_check["outcome"].isel(round=[3, 4]).max("fortune"),
    ],
    dim="round",
)

biting_words_damage = damage(
    biting_words_check, Damage("sonic", biting_words_rank * 2, 6)
).total_damage
biting_words_damage.mean("roll").stack(
    col=["round", "incapacitation_spell"]
).to_pandas()

### Put it all together

In [ ]:
final = xarray.Dataset(
    {
        "AC": AC,
        "Will": will,
        "off_guard": blindness_paralyze.off_guard,
        "need_evil_eye": blindness_paralyze.need_evil_eye,
        "blistering_invective": blistering_invective_damage,
        "biting_words": biting_words_damage,
        "total_damage": blistering_invective_damage + biting_words_damage,
    }
).transpose("target", "roll", "round", "incapacitation_spell")
final["harmed"] = final.total_damage.sum("round") > 0
final["bloodied"] = final.total_damage.sum("round") > targets.HP // 2
final["killed"] = final.total_damage.sum("round") >= targets.HP
final

### Let's analyse our results!
#### Mean cumulative damage by the end of the attack routine

In [ ]:
(
    final[["blistering_invective", "biting_words", "total_damage"]]
    .mean("roll")
    .sum("round")
    .to_array("component")
    .stack(col=["component", "incapacitation_spell"])
    .to_pandas()
)

- Probability of dealing any HP damage at all
- Probability of dealing more than 50% HP damage
- Probability of solo killing the target
- Probability of blinding the target in round 2
- Probability of needing to spam evil eye every round to keep them blind

In [ ]:
(
    final[["harmed", "bloodied", "killed", "off_guard", "need_evil_eye"]]
    .mean("roll")
    .to_array("condition")
    .stack(col=["condition", "incapacitation_spell"])
    .to_pandas()
)

#### Damage distribution

In [ ]:
_ = (
    final["total_damage"]
    .sum("round")
    .sel(incapacitation_spell="blindness")
    .T.to_pandas()
    .hist(bins=50, figsize=(10, 8))
)

#### Damage distribution, normalized by target's hit points total

In [ ]:
_ = (
    (final["total_damage"].sum("round") / targets.HP)
    .sel(incapacitation_spell="blindness")
    .T.to_pandas()
    .hist(bins=50, figsize=(10, 8))
)

#### Worst AC over the 5 rounds

In [ ]:
_ = (
    final["AC"]
    .min("round")
    .sel(incapacitation_spell="blindness")
    .T.to_pandas()
    .hist(bins=20, figsize=(10, 8))
)

#### Worst Will bonus over the 5 rounds

In [ ]:
_ = (
    final["Will"]
    .min("round")
    .sel(incapacitation_spell="blindness")
    .T.to_pandas()
    .hist(bins=20, figsize=(10, 8))
)